# Exploring the Strava Graph with Py2neo

In this notebook we'll look at how to use [v4 of Py2neo](https://medium.com/neo4j/py2neo-v4-2bedc8afef2) to query a Neo4j database that contains running data from the Strava API.

First we'll import the libray and create a connection to a database running locally:

In [1]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo"))

## Find the most recent activities

Let's start by finding the 10 most recent runs along with the distance, time, and pace for each of them.
The following query returns this information:

In [2]:
graph.run("""\
MATCH (r:Run)
WITH r { .id, .startDate, .name, .movingTime, .distance,
           pace: duration({seconds: r.movingTime.seconds / r.distance * 1609.34})
         }
RETURN r.name, 
       apoc.date.format(r.startDate.epochSeconds, 's', 'MMM d yyyy') AS dateOfRun,
       r.distance, 
       apoc.date.format(r.movingTime.milliseconds, 'ms', 'HH:mm:ss') AS time,
       apoc.date.format(r.pace.milliseconds, "ms", "mm:ss") AS overallPace
ORDER BY r.startDate DESC
LIMIT 10
""").to_table()

r.name,dateOfRun,r.distance,time,overallPace
Morning Run,Jun 13 2018,10780.2,00:49:11,07:20
Morning Run,Jun 11 2018,10035.8,00:45:53,07:21
Morning Run,Jun 9 2018,11043.2,00:48:29,07:03
Morning Run,Jun 8 2018,11281.2,00:51:57,07:24
Morning Run,Jun 6 2018,10884.9,00:48:38,07:11
Morning Run,Jun 4 2018,10030.5,00:44:14,07:05
Morning Run,Jun 2 2018,13039.8,00:57:41,07:07
Morning Run,Jun 1 2018,10701.7,00:48:02,07:13
Morning Run,May 30 2018,9828.4,00:46:06,07:32
Morning Run,May 28 2018,10193.0,00:46:42,07:22


## Find the most recent activities in 2017

What if we want the find activities for a particular year?

In [3]:
graph.run("""\
MATCH (r:Run)
WHERE r.startDate.year = 2017
WITH r { .id, .startDate, .name, .movingTime, .distance,
           pace: duration({seconds: r.movingTime.seconds / r.distance * 1609.34})
         }
RETURN r.name, 
       apoc.date.format(r.startDate.epochSeconds, 's', 'MMM d yyyy') AS dateOfRun,
       r.distance, 
       apoc.date.format(r.movingTime.milliseconds, 'ms', 'HH:mm:ss') AS time,
       apoc.date.format(r.pace.milliseconds, "ms", "mm:ss") AS overallPace
ORDER BY r.startDate DESC
LIMIT 10
""").to_table()

r.name,dateOfRun,r.distance,time,overallPace
Evening Run,Dec 30 2017,7773.6,00:36:34,07:34
Morning Run,Dec 29 2017,10528.1,00:49:57,07:38
Lunch Run,Dec 27 2017,8579.2,00:41:08,07:42
Morning Run,Dec 25 2017,8093.7,00:38:32,07:39
Evening Run,Dec 23 2017,8094.3,00:38:58,07:44
Lunch Run,Dec 22 2017,7547.1,00:36:23,07:45
Morning Run,Dec 20 2017,8106.8,00:38:47,07:41
Morning Run,Dec 18 2017,8070.3,00:40:43,08:07
Afternoon Run,Dec 16 2017,8497.1,00:41:02,07:46
Morning Run,Dec 15 2017,7613.0,00:36:01,07:36


## What was the longest distance run each year?

In [4]:
graph.run("""\
MATCH (r:Run)
WITH r
ORDER BY r.startDate.year, r.distance DESC

WITH r.startDate.year AS year, collect(r)[0] AS longestRun

WITH year, 
     longestRun { .id, .startDate, .name, .movingTime, .distance,
                  pace: duration({seconds: longestRun.movingTime.seconds / longestRun.distance * 1609.34})
                }
RETURN year,
       longestRun.name, 
       apoc.date.format(longestRun.startDate.epochSeconds, 's', 'MMM d yyyy') AS dateOfRun,
       longestRun.distance, 
       apoc.date.format(longestRun.movingTime.milliseconds, 'ms', 'HH:mm:ss') AS time,
       apoc.date.format(longestRun.pace.milliseconds, "ms", "mm:ss") AS overallPace
ORDER BY year       
""").to_table()

year,longestRun.name,dateOfRun,longestRun.distance,time,overallPace
2014,Morning Run,Dec 3 2014,4703.9,00:28:18,09:40
2015,Morning Run,Dec 31 2015,9497.3,00:53:34,09:04
2016,Morning Run,Jan 2 2016,10457.9,00:56:57,08:45
2017,Morning Run,Jan 22 2017,10764.1,00:56:47,08:29
2018,Morning Run,May 19 2018,13635.2,01:01:27,07:15


## What was the quickest run each year?

In [5]:
graph.run("""\
MATCH (r:Run)
WITH r { .id, .startDate, .name, .movingTime, .distance,
         pace: duration({seconds: r.movingTime.seconds / r.distance * 1609.34})
       }
ORDER BY r.startDate.year, r.pace

WITH r.startDate.year AS year, collect(r)[0] AS longestRun

RETURN year,
       longestRun.name, 
       apoc.date.format(longestRun.startDate.epochSeconds, 's', 'MMM d yyyy') AS dateOfRun,
       longestRun.distance, 
       apoc.date.format(longestRun.movingTime.milliseconds, 'ms', 'HH:mm:ss') AS time,
       apoc.date.format(longestRun.pace.milliseconds, "ms", "mm:ss") AS overallPace
ORDER BY year       
""").to_table()

year,longestRun.name,dateOfRun,longestRun.distance,time,overallPace
2014,Morning Run,Nov 29 2014,4136.0,00:22:20,08:41
2015,Evening Run,Jul 5 2015,4360.9,00:20:31,07:34
2016,Morning Run,Jul 22 2016,4147.5,00:18:30,07:10
2017,Morning Run,Mar 26 2017,5355.8,00:24:42,07:25
2018,Morning Run,Jun 9 2018,11043.2,00:48:29,07:03


## What were my quickest estimated runs?

In our graph we also have estimated attempts at different distances. 
For example, these are my best estimated 10k attempts:

In [6]:
estimated_effort_query = """\
MATCH (distance:Distance {name: {distance}})<-[:DISTANCE]-(effort)<-[:DISTANCE_EFFORT]-(run)

WITH run { .id, .startDate, .distance,
           pace: duration({seconds: run.elapsedTime.seconds / run.distance * 1609.34})
         },
     effort { .elapsedTime,
              pace: duration({seconds: effort.elapsedTime.seconds / distance.distance * 1609.34  })
            }

RETURN run.id,
       apoc.date.format(run.startDate.epochSeconds, 's', 'MMM d yyyy') AS dateOfRun,
       apoc.date.format(effort.elapsedTime.milliseconds, 'ms', 'mm:ss') AS time,
       apoc.date.format(effort.pace.milliseconds, "ms", "mm:ss") AS pace,
       apoc.date.format(run.pace.milliseconds, "ms", "mm:ss") AS overallPace,
       run.distance AS totalDistance 
ORDER BY effort.elapsedTime
LIMIT {limit}
"""

graph.run(estimated_effort_query, {"distance": "10k", "limit": 10}).to_table()

run.id,dateOfRun,time,pace,overallPace,totalDistance
1626636004,Jun 9 2018,44:05,07:05,07:05,11043.2
1611168962,Jun 2 2018,44:12,07:06,07:07,13039.8
1616050069,Jun 4 2018,44:35,07:10,07:10,10030.5
1620188065,Jun 6 2018,44:49,07:12,07:11,10884.9
1609355524,Jun 1 2018,44:59,07:14,07:13,10701.7
1581401227,May 19 2018,45:02,07:14,07:17,13635.2
1550422417,May 5 2018,45:26,07:18,07:20,11674.4
1585759604,May 21 2018,45:41,07:21,07:20,10147.0
1635264102,Jun 13 2018,45:48,07:22,07:21,10780.2
1575082905,May 16 2018,45:49,07:22,07:22,10148.8


We can run the same query passing in a different value to the `distance` parameter:

In [7]:
graph.run(estimated_effort_query, {"distance": "5k", "limit": 10}).to_table()

run.id,dateOfRun,time,pace,overallPace,totalDistance
1611168962,Jun 2 2018,21:24,06:53,07:07,13039.8
1626636004,Jun 9 2018,21:34,06:56,07:05,11043.2
1366081623,Jan 20 2018,21:55,07:03,07:16,8684.0
1620188065,Jun 6 2018,22:00,07:04,07:11,10884.9
1616050069,Jun 4 2018,22:01,07:05,07:10,10030.5
1475714025,Mar 28 2018,22:15,07:09,07:12,6949.4
1581401227,May 19 2018,22:15,07:09,07:17,13635.2
1609355524,Jun 1 2018,22:16,07:10,07:13,10701.7
1485759514,Apr 2 2018,22:20,07:11,07:14,7772.0
1559310491,May 9 2018,22:20,07:11,07:14,9019.7


In [8]:
graph.run(estimated_effort_query, {"distance": "1k", "limit": 10}).to_table()

run.id,dateOfRun,time,pace,overallPace,totalDistance
1620188065,Jun 6 2018,03:58,06:23,07:11,10884.9
914796614,Mar 26 2017,04:08,06:39,07:26,5355.8
1544469717,May 2 2018,04:08,06:39,07:27,8560.3
1626636004,Jun 9 2018,04:08,06:39,07:05,11043.2
1366081623,Jan 20 2018,04:10,06:42,07:16,8684.0
1485759514,Apr 2 2018,04:10,06:42,07:14,7772.0
1609355524,Jun 1 2018,04:10,06:42,07:13,10701.7
1611168962,Jun 2 2018,04:10,06:42,07:07,13039.8
1594415412,May 25 2018,04:11,06:43,07:27,11690.0
1563554226,May 11 2018,04:12,06:45,07:28,9084.8


# Combining estimated runs

What if I want to combine my efforts for multiple distances in a single row? 
Instead of returning a table of results we can use the `to_data_frame` function to return a Pandas DataFrame and then join those DataFrames:

In [9]:
import pandas as pd

In [18]:
df_5k = graph.run(estimated_effort_query, {"distance": "5k", "limit": 1000}).to_data_frame()
df_10k = graph.run(estimated_effort_query, {"distance": "10k", "limit": 1000}).to_data_frame()

pd.merge(df_5k, df_10k, on="run.id")[["run.id", "dateOfRun_x", "time_x", "pace_x", "time_y", "pace_y"]].head(10)

,run.id,dateOfRun_x,time_x,pace_x,time_y,pace_y
0,1611168962,Jun 2 2018,21:24,06:53,44:12,07:06
1,1626636004,Jun 9 2018,21:34,06:56,44:05,07:05
2,1620188065,Jun 6 2018,22:00,07:04,44:49,07:12
3,1616050069,Jun 4 2018,22:01,07:05,44:35,07:10
4,1581401227,May 19 2018,22:15,07:09,45:02,07:14
5,1609355524,Jun 1 2018,22:16,07:10,44:59,07:14
6,1635264102,Jun 13 2018,22:20,07:11,45:48,07:22
7,1585759604,May 21 2018,22:27,07:13,45:41,07:21
8,1600870897,May 28 2018,22:30,07:14,45:51,07:22
9,1566256778,May 12 2018,22:30,07:14,46:11,07:25


In [11]:
df_1k = graph.run(estimated_effort_query, {"distance": "1k", "limit": 1000}).to_data_frame()
df_10k = graph.run(estimated_effort_query, {"distance": "10k", "limit": 1000}).to_data_frame()

cols = ["run.id", "dateOfRun_x", "time_x", "pace_x", "time_y", "pace_y"]
df_1k_10k = pd.merge(df_1k, df_10k, on="run.id")[cols]

In [12]:
df_1k_10k.sort_values(by=['time_y']).head(10)

,run.id,dateOfRun_x,time_x,pace_x,time_y,pace_y
1,1626636004,Jun 9 2018,04:08,06:39,44:05,07:05
3,1611168962,Jun 2 2018,04:10,06:42,44:12,07:06
7,1616050069,Jun 4 2018,04:16,06:51,44:35,07:10
0,1620188065,Jun 6 2018,03:58,06:23,44:49,07:12
2,1609355524,Jun 1 2018,04:10,06:42,44:59,07:14
13,1581401227,May 19 2018,04:19,06:56,45:02,07:14
6,1550422417,May 5 2018,04:14,06:48,45:26,07:18
18,1585759604,May 21 2018,04:22,07:01,45:41,07:21
5,1635264102,Jun 13 2018,04:13,06:47,45:48,07:22
11,1575082905,May 16 2018,04:17,06:53,45:49,07:22


In [13]:
df_1k_10k.sort_values(by=['time_x']).head(10)

,run.id,dateOfRun_x,time_x,pace_x,time_y,pace_y
0,1620188065,Jun 6 2018,03:58,06:23,44:49,07:12
1,1626636004,Jun 9 2018,04:08,06:39,44:05,07:05
2,1609355524,Jun 1 2018,04:10,06:42,44:59,07:14
3,1611168962,Jun 2 2018,04:10,06:42,44:12,07:06
4,1594415412,May 25 2018,04:11,06:43,46:22,07:27
5,1635264102,Jun 13 2018,04:13,06:47,45:48,07:22
6,1550422417,May 5 2018,04:14,06:48,45:26,07:18
7,1616050069,Jun 4 2018,04:16,06:51,44:35,07:10
8,1566256778,May 12 2018,04:16,06:51,46:11,07:25
9,1631066785,Jun 11 2018,04:17,06:53,45:54,07:23


In [14]:
df_2mile = graph.run(estimated_effort_query, {"distance": "2 mile", "limit": 1000}).to_data_frame()
df_10k = graph.run(estimated_effort_query, {"distance": "10k", "limit": 1000}).to_data_frame()

cols = ["run.id", "dateOfRun_x", "time_x", "pace_x", "time_y", "pace_y"]
df_2mile_10k = pd.merge(df_2mile, df_10k, on="run.id")[cols]

In [15]:
df_2mile_10k.sort_values(by=['time_x']).head(10)

,run.id,dateOfRun_x,time_x,pace_x,time_y,pace_y
0,1620188065,Jun 6 2018,13:38,06:48,44:49,07:12
1,1626636004,Jun 9 2018,13:42,06:50,44:05,07:05
2,1611168962,Jun 2 2018,13:47,06:53,44:12,07:06
3,1616050069,Jun 4 2018,14:08,07:03,44:35,07:10
4,1581401227,May 19 2018,14:13,07:06,45:02,07:14
5,1609355524,Jun 1 2018,14:14,07:06,44:59,07:14
6,1594415412,May 25 2018,14:15,07:07,46:22,07:27
7,1575082905,May 16 2018,14:17,07:08,45:49,07:22
8,1631066785,Jun 11 2018,14:17,07:08,45:54,07:23
9,1579256510,May 18 2018,14:18,07:08,46:19,07:27


In [16]:
multiple_efforts_query = """\
MATCH (distance10k:Distance {name: {distance1}})<-[:DISTANCE]-(effort10k)<-[:DISTANCE_EFFORT]-(run),
      (distance5k:Distance {name: {distance2}})<-[:DISTANCE]-(effort5k)<-[:DISTANCE_EFFORT]-(run)

WITH run { .id, .startDate, .distance,
           pace: duration({seconds: run.elapsedTime.seconds / run.distance * 1609.34})
         },
     effort10k { .elapsedTime,
              pace: duration({seconds: effort10k.elapsedTime.seconds / distance10k.distance * 1609.34  })
            },
     effort5k { .elapsedTime,
              pace: duration({seconds: effort5k.elapsedTime.seconds / distance5k.distance * 1609.34  })
            }            
      
RETURN apoc.date.format(run.startDate.epochSeconds, 's', 'MMM d yyyy') AS dateOfRun,
       apoc.date.format(effort5k.elapsedTime.milliseconds, 'ms', 'mm:ss') AS time5k,
       apoc.date.format(effort5k.pace.milliseconds, "ms", "mm:ss") AS pace5k,
apoc.date.format(effort10k.elapsedTime.milliseconds, 'ms', 'mm:ss') AS time10k,

       apoc.date.format(effort10k.pace.milliseconds, "ms", "mm:ss") AS pace10k,
       run.distance AS totalDistance 
ORDER BY effort5k.pace
"""

graph.run(multiple_efforts_query, {"distance1": "10k", "distance2": "5k"}).to_table()

dateOfRun,time5k,pace5k,time10k,pace10k,totalDistance
Jun 2 2018,21:24,06:53,44:12,07:06,13039.8
Jun 9 2018,21:34,06:56,44:05,07:05,11043.2
Jun 6 2018,22:00,07:04,44:49,07:12,10884.9
Jun 4 2018,22:01,07:05,44:35,07:10,10030.5
May 19 2018,22:15,07:09,45:02,07:14,13635.2
Jun 1 2018,22:16,07:10,44:59,07:14,10701.7
Jun 13 2018,22:20,07:11,45:48,07:22,10780.2
May 21 2018,22:27,07:13,45:41,07:21,10147.0
Jun 8 2018,22:30,07:14,46:11,07:25,11281.2
May 12 2018,22:30,07:14,46:11,07:25,12963.9


In [17]:
graph.run("MATCH (r:Run) RETURN r {.id, .name} LIMIT 2", {"distance1": "10k", "distance2": "5k"})